論文<br>
<br>
<br>
GitHub<br>
https://github.com/albarji/mixture-of-diffusers<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/MixtureDiffUser_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/albarji/mixture-of-diffusers.git

# using Commits on Feb 12, 2023
%cd /content/mixture-of-diffusers
!git checkout a0a1ac6d71aaa91e0aec73336eeb7ddc4f08fe21

## ライブラリのインストール

In [ ]:
%cd /content/mixture-of-diffusers

!pip install diffusers[torch]==0.7.* ftfy==6.1.* gitpython==3.1.* ligo-segments==1.4.* transformers==4.21.*

## ライブラリのインポート

In [ ]:
%cd /content/mixture-of-diffusers

from diffusers import LMSDiscreteScheduler
from mixdiff.tiling import (
    StableDiffusionTilingPipeline, 
    StableDiffusionCanvasPipeline, Text2ImageRegion)

import torch
device = 'cuda' if torch.cuda.is_available() else "cpu"
print("using device is", device)

access_token = "ここにアクセストークンを設定"

# TilingPipeling

## 学習済みモデルのセットアップ

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"

# load scheduler
scheduler = LMSDiscreteScheduler(
    beta_start=0.00085, 
    beta_end=0.012, 
    beta_schedule="scaled_linear", 
    num_train_timesteps=1000)

# load model
pipeline = StableDiffusionTilingPipeline.from_pretrained(
    model_id, 
    scheduler = scheduler, 
    use_auth_token = access_token
    ).to(device)

In [ ]:
prompt = [
    "a tokyo city, ukiyo-e, intricate, elegant, highly detailed, smooth, sharp focus, artstation, stunning masterpiece, impressive colors",
    "a tokyo city, Van Gogh, intricate, elegant, highly detailed, smooth, sharp focus, artstation, stunning masterpiece, impressive colors",
    "a tokyo city, modern anime, intricate, elegant, highly detailed, smooth, sharp focus, artstation, stunning masterpiece, impressive colors",
    "a tokyo city, Banksy, intricate, elegant, highly detailed, smooth, sharp focus, artstation, stunning masterpiece, impressive colors",
]

In [ ]:
# num_inference_steps: number of diffusions steps.
# guidance_scale: classifier-free guidance.
# seed: general random seed to initialize latents.
# tile_height: height in pixels of each grid tile.
# tile_width: width in pixels of each grid tile.
# tile_row_overlap: number of overlap pixels between tiles in consecutive rows.
# tile_col_overlap: number of overlap pixels between tiles in consecutive columns.
# guidance_scale_tiles: specific weights for classifier-free guidance in each tile.
# guidance_scale_tiles: specific weights for classifier-free guidance in each tile. If None, the value provided in guidance_scale will be used.
# seed_tiles: specific seeds for the initialization latents in each tile. These will override the latents generated for the whole canvas using the standard seed parameter.
# seed_tiles_mode: either "full" "exclusive". If "full", all the latents affected by the tile be overriden. If "exclusive", only the latents that are affected exclusively by this tile (and no other tiles) will be overrriden.
# seed_reroll_regions: a list of tuples in the form (start row, end row, start column, end column, seed) defining regions in pixel space for which the latents will be overriden using the given seed. Takes priority over seed_tiles.

image = pipeline(
    prompt=[
        prompt
    ],
    tile_height = 640,
    tile_width = 640,
    tile_row_overlap = 0,
    tile_col_overlap = 256,
    guidance_scale = 8,
    seed = 12,
    num_inference_steps = 50,
)["sample"][0]

image

# CanvasPipeline

## 学習済みモデルのセットアップ

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"

# load scheduler
scheduler = LMSDiscreteScheduler(
    beta_start=0.00085, 
    beta_end=0.012, 
    beta_schedule="scaled_linear", 
    num_train_timesteps=1000)

# load model
pipeline = StableDiffusionCanvasPipeline.from_pretrained(
    model_id, 
    scheduler = scheduler, 
    use_auth_token = access_token
    ).to(device)


In [ ]:
image = pipeline(
    canvas_height=640,
    canvas_width=1408,
    regions=[
        Text2ImageRegion(0, 640, 0, 640, guidance_scale=8,
            prompt=f"A charming house in the countryside, by jakub rozalski, sunset lighting, elegant, highly detailed, smooth, sharp focus, artstation, stunning masterpiece"),
        Text2ImageRegion(0, 640, 384, 1024, guidance_scale=8,
            prompt=f"A dirt road in the countryside crossing pastures, by jakub rozalski, sunset lighting, elegant, highly detailed, smooth, sharp focus, artstation, stunning masterpiece"),
        Text2ImageRegion(0, 640, 768, 1408, guidance_scale=8,
            prompt=f"An old and rusty giant robot lying on a dirt road, by jakub rozalski, dark sunset lighting, elegant, highly detailed, smooth, sharp focus, artstation, stunning masterpiece"),
    ],
    num_inference_steps=50,
    seed=7178915308,
)["sample"][0]

image